In [8]:
import pandas as pd 
import numpy as np
import re


import da base

In [9]:
base=pd.read_csv('regex.csv')

na base temos esse layout basico onde temos um protocolo e um campo mensagem onde o operador escreve em um campo texto o novo email, novo telefone

temos 2 problemas: 
1 o protocolo pode estar quebrado em  varias linhas 
2 extrair numeros e o email dester campos


In [10]:
base

,PROTOCOLO,MENSAGEM
0,111111,"TROCA DE EMAIL , EMAIL NOVO ABC@GMAIL.COM"
1,111111,EMAIL ANTIGO ABC1@GMAIL.COM
2,111112,# TEL (22)9994-9999
3,111113,EMAIL NOVO TRRRRR@EMAIL.COM EMAAIL ANTIGO RTTT...
4,111113,TELEFONE NOVO 119999-9999
5,111113,# TEL (99)9993-9995
6,111114,FONE ANTIGO 119888899 PARA 210999999
7,111115,"TROCA DE EMAIL , EMAIL NOVO NEWEAMIL@GMAIL.COM"


primeiro segmentamos os valores em lista 

In [11]:
df=base.groupby(['PROTOCOLO'])['MENSAGEM'].apply(lambda x:x.values.flatten() )
melt=pd.DataFrame(df.values.tolist(),index=df.index).add_prefix('mensagem_')

In [12]:
melt

,mensagem_0,mensagem_1,mensagem_2
PROTOCOLO,,,
111111,"TROCA DE EMAIL , EMAIL NOVO ABC@GMAIL.COM",EMAIL ANTIGO ABC1@GMAIL.COM,None
111112,# TEL (22)9994-9999,None,None
111113,EMAIL NOVO TRRRRR@EMAIL.COM EMAAIL ANTIGO RTTT...,TELEFONE NOVO 119999-9999,# TEL (99)9993-9995
111114,FONE ANTIGO 119888899 PARA 210999999,None,None
111115,"TROCA DE EMAIL , EMAIL NOVO NEWEAMIL@GMAIL.COM",None,None


concatenamos os campos 

In [13]:
melt['descricao']=melt['mensagem_0'].astype(str)+' * '+ melt['mensagem_1'].astype(str)+' * '+melt['mensagem_2'].astype(str)

indexando a base

In [14]:
base.set_index('PROTOCOLO')

,MENSAGEM
PROTOCOLO,
111111,"TROCA DE EMAIL , EMAIL NOVO ABC@GMAIL.COM"
111111,EMAIL ANTIGO ABC1@GMAIL.COM
111112,# TEL (22)9994-9999
111113,EMAIL NOVO TRRRRR@EMAIL.COM EMAAIL ANTIGO RTTT...
111113,TELEFONE NOVO 119999-9999
111113,# TEL (99)9993-9995
111114,FONE ANTIGO 119888899 PARA 210999999
111115,"TROCA DE EMAIL , EMAIL NOVO NEWEAMIL@GMAIL.COM"


juntando a base 

In [15]:
base_junta=base.join(melt,on='PROTOCOLO')
base_junta

,PROTOCOLO,MENSAGEM,mensagem_0,mensagem_1,mensagem_2,descricao
0,111111,"TROCA DE EMAIL , EMAIL NOVO ABC@GMAIL.COM","TROCA DE EMAIL , EMAIL NOVO ABC@GMAIL.COM",EMAIL ANTIGO ABC1@GMAIL.COM,None,"TROCA DE EMAIL , EMAIL NOVO ABC@GMAIL.COM * EM..."
1,111111,EMAIL ANTIGO ABC1@GMAIL.COM,"TROCA DE EMAIL , EMAIL NOVO ABC@GMAIL.COM",EMAIL ANTIGO ABC1@GMAIL.COM,None,"TROCA DE EMAIL , EMAIL NOVO ABC@GMAIL.COM * EM..."
2,111112,# TEL (22)9994-9999,# TEL (22)9994-9999,None,None,# TEL (22)9994-9999 * nan * nan
3,111113,EMAIL NOVO TRRRRR@EMAIL.COM EMAAIL ANTIGO RTTT...,EMAIL NOVO TRRRRR@EMAIL.COM EMAAIL ANTIGO RTTT...,TELEFONE NOVO 119999-9999,# TEL (99)9993-9995,EMAIL NOVO TRRRRR@EMAIL.COM EMAAIL ANTIGO RTTT...
4,111113,TELEFONE NOVO 119999-9999,EMAIL NOVO TRRRRR@EMAIL.COM EMAAIL ANTIGO RTTT...,TELEFONE NOVO 119999-9999,# TEL (99)9993-9995,EMAIL NOVO TRRRRR@EMAIL.COM EMAAIL ANTIGO RTTT...
5,111113,# TEL (99)9993-9995,EMAIL NOVO TRRRRR@EMAIL.COM EMAAIL ANTIGO RTTT...,TELEFONE NOVO 119999-9999,# TEL (99)9993-9995,EMAIL NOVO TRRRRR@EMAIL.COM EMAAIL ANTIGO RTTT...
6,111114,FONE ANTIGO 119888899 PARA 210999999,FONE ANTIGO 119888899 PARA 210999999,None,None,FONE ANTIGO 119888899 PARA 210999999 * nan * nan
7,111115,"TROCA DE EMAIL , EMAIL NOVO NEWEAMIL@GMAIL.COM","TROCA DE EMAIL , EMAIL NOVO NEWEAMIL@GMAIL.COM",None,None,"TROCA DE EMAIL , EMAIL NOVO NEWEAMIL@GMAIL.COM..."


dropando as colunas 

In [16]:
base_junta.drop(['MENSAGEM','mensagem_0','mensagem_1','mensagem_2'],axis=1, inplace=True)


retirando duplicidade

In [93]:
base_junta.drop_duplicates(keep='first',inplace=True)



normalizando a base 

In [94]:

base_junta['descricao']=base_junta['descricao'].str.replace('(',' ')
base_junta['descricao']=base_junta['descricao'].str.replace(')','')

base_junta['descricao']=base_junta['descricao'].str.replace('-','')


data=pd.DataFrame()

In [95]:
base_junta

,PROTOCOLO,descricao
0,111111,"TROCA DE EMAIL , EMAIL NOVO ABC@GMAIL.COM * EM..."
2,111112,# TEL 2299949999 * nan * nan
3,111113,EMAIL NOVO TRRRRR@EMAIL.COM EMAAIL ANTIGO RTTT...
6,111114,FONE ANTIGO 119888899 PARA 210999999 * nan * nan
7,111115,"TROCA DE EMAIL , EMAIL NOVO NEWEAMIL@GMAIL.COM..."


aplicando os regex 

In [99]:
for val in base_junta.descricao:
    email=re.findall(r'[\w\.-]+@[\w\.-]+(?:\.[\w]+)+',val)
    phone=re.findall(r'\d+',val)
    print(phone)
    print(email)
    

['1']
['ABC@GMAIL.COM', 'ABC1@GMAIL.COM']
['2299949999']
[]
['1199999999', '9999939995']
['TRRRRR@EMAIL.COM', 'RTTTTT@EMAIL.COM']
['119888899', '210999999']
[]
[]
['NEWEAMIL@GMAIL.COM']
